In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
credits.head()

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
credits.head(1)['crew'].values

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.info()

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
import ast

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L=[]
    count = 0
    for i in ast.literal_eval(obj):
        if count!=3:
            L.append(i['name'])
            count+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
type(movies.iloc[0].overview)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['tags'].apply(lambda x:" ".join(x))

In [ ]:
movies.head()

In [ ]:
movies['tags'][0]

In [ ]:
movies['tags'] = movies['tags'].apply(lambda x:x.lower())

In [ ]:
movies['tags'][0]

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)
    

In [ ]:
movies['tags'] = movies['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(movies['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(movies.iloc[i[0]].title)


In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle

In [ ]:
pickle.dump(movies,open('movies.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
similarity

In [ ]:
movies['title']